## Aplicando Web scraping a la página uniprot

### Presentado por:
* Chucas Mera, Ana Lisset
* Huerta Barba, Stefano 
* Cuellar Tolentino, Elian 

### Objetivo
Extraer la información de la pagina web Uniprot de cada gen para posteriormente almacenarlos en un archivo excel.
#### Nota:
<span style="color:blue;"> 
hemos reducido el excel original a 10 genes por hoja, para que sea menos pesado al subir</span>

In [4]:
# Librerias
import pandas as pd                         # Creación de tablas
import numpy as np
import openpyxl                             # Leer archivos xlsx
import requests                             # Decodifición de archivos HTTP
from bs4 import BeautifulSoup as b          # Extración de archivos de HTML
import re                                   # Codificación de expresiones regulares
from openpyxl import load_workbook          # Inicador de un trabajo en excel

##### PRIMERO: 	
El excel **"02 Ch Genes ordenados con vecinos.xlsx"** está compuesta por varias hojas, por lo que vamos a realizar la lectura del excel y como nos interesa la columna con etiqueta: **"GenAbrev"** y a partir de ella poder buscar en la página, vamos a realizar la extración de la columna tal como se muestra a continuación:

In [13]:
# Lectura de datos
data = pd.read_excel("02 Ch Genes ordenados con vecinos.xlsx",sheet_name=None,header = 1)
data.keys() # lista de los nombres de cada hoja de excel

dict_keys(['C1 Milagros', 'C2 Wilmer', 'C3 Gisell', 'C4 Jerica', 'C5 Manuel', 'C6 Jerica', 'C7 Mariana', 'C8 Rosha', 'C9 Jerica', 'C10 Lucero', 'C11 Nicole', 'C12 Gisell', 'C13 Lucero', 'C14 Wilmer', 'C15Milagros', 'C16', 'C17', 'C18'])

In [6]:
# Unimos los datos de cada excel en un solo objeto
data1 = pd.concat(data,axis = 0)
data1

Bt    N° N° Articulos  Sel     Gen Abrev Category  \
C1 Milagros 0  NaN   1.0          NaN  NaN     TRNAS-GGA      NaN   
            1  NaN   2.0          NaN  NaN  LOC102173449   pseudo   
            2  NaN   3.0          NaN  NaN  LOC102169333   pseudo   
            3  NaN   4.0          NaN  NaN  LOC108636757    ncRNA   
            4  NaN   5.0          NaN  NaN  LOC102187712   pseudo   
...            ...   ...          ...  ...           ...      ...   
C18         5  NaN   6.0          NaN  NaN         AKTIP      NaN   
            6  NaN   7.0          NaN  NaN  LOC102182337      NaN   
            7  NaN   8.0          NaN  NaN      RPGRIP1L      NaN   
            8  NaN   9.0          NaN  NaN           FTO      NaN   
            9  NaN  10.0          NaN  NaN     TRNAG-UCC      NaN   

                                                Name of the gene    Inicio  \
C1 Milagros 0                                                NaN  32091689   
            1                        olfactory receptor 6C2-like  32837131   
            2   probable palmitoyltransferase ZDHHC16 pseudogene  33089112   
            3                       uncharacterized LOC108636757  33764252   
            4  eukaryotic translation initiation factor 3 sub...  33810174   
...                                                          ...       ...   
C18         5                                                NaN  23217384   
            6                                                NaN  23305711   
            7                                                NaN  23318073   
            8                                                NaN  23418852   
            9                                                NaN  23514242   

                  Final RNASeq  ... Unnamed: 38 Unnamed: 39 Unnamed: 40  \
C1 Milagros 0  32091761    NaN  ...         NaN         NaN         NaN   
            1  32838076    NaN  ...         NaN         NaN         NaN   
            2  33090132    NaN  ...         NaN         NaN         NaN   
            3  33768221    NaN  ...         NaN         NaN         NaN   
            4  33811741    NaN  ...         NaN         NaN         NaN   
...                 ...    ...  ...         ...         ...         ...   
C18         5  23227007    NaN  ...         NaN         NaN         NaN   
            6  23306648    NaN  ...         NaN         NaN         NaN   
            7  23418624    NaN  ...         NaN         NaN         NaN   
            8  23845211    NaN  ...         NaN         NaN         NaN   
            9  23514314    NaN  ...         NaN         NaN         NaN   

              Unnamed: 41 Unnamed: 42 Unnamed: 43 Unnamed: 44 Unnamed: 17  \
C1 Milagros 0         NaN         NaN         NaN         NaN         NaN   
            1         NaN         NaN         NaN         NaN         NaN   
            2         NaN         NaN         NaN         NaN         NaN   
            3         NaN         NaN         NaN         NaN         NaN   
            4         NaN         NaN         NaN         NaN         NaN   
...                   ...         ...         ...         ...         ...   
C18         5         NaN         NaN         NaN         NaN         NaN   
            6         NaN         NaN         NaN         NaN         NaN   
            7         NaN         NaN         NaN         NaN         NaN   
            8         NaN         NaN         NaN         NaN         NaN   
            9         NaN         NaN         NaN         NaN         NaN   

               Genecards (human)   BT  
C1 Milagros 0                NaN  NaN  
            1                NaN  NaN  
            2                NaN  NaN  
            3                NaN  NaN  
            4                NaN  NaN  
...                          ...  ...  
C18         5                NaN  NaN  
            6                NaN  NaN  
            7                NaN  NaN  
            8           

In [1]:
#data1.info()

In [9]:
#Extracción de la columna "Gen Abrev" de cada una de las hojas de excel
gene=data1["Gen Abrev"]
gene=gene.tolist()      # Lo tranformamos en una lista
genes=gene[0:20] 
genes

['TRNAS-GGA',
 'LOC102173449',
 'LOC102169333',
 'LOC108636757',
 'LOC102187712',
 'VGLL3',
 'CHMP2B',
 'POU1F1',
 'HTR1F',
 'LOC106503629',
 'C2H1orf64',
 'CLCNKA',
 'HSPB7',
 'FAM131C',
 'LOC102181451',
 'EPHA2',
 'LOC108637009',
 'ARHGEF19',
 'RSG1',
 'LOC106503868']

In [14]:
# Aquí creamos la variable "encabezados" con los nombres de los datos que vamos a extraer de la página, como lo indica el word.
encabezados=[['Gen', 'Entry', 'Entryname' , 'Protein', 'Gene', 'Organism', 
                 'Status', 'GO_Biological_process']]

#### SEGUNDO
Luego de poder tener en la variable **"genes"** una lista creada a partir de la columna "GenAbrev", hemos explorado la página a travéz del inspector de google para poder hallar algunos patrones y sobre todo la raiz y las hijas de la raiz, por lo que hemos creado un bucle for para poder facilitar la busqueda dentro de la página.

En la primera parte del for creamos el URL de busqueda a partir del URL base y tambien usamos mucho las librerias **requests** para decodificar los archivos HTTP y **BeautifulSoup**  para poder extraer einteractuar con los elementos de la página Uniprot

<span style="color:blue;"> 
Nota: pueden seguir nuestros comentarios para un mejor entendimiento del código</span>

In [15]:
for Gen in genes: 
    #Creamos el  url con cada gen agregando "capra hircus" en un ciclo for de la lista "gen lista"
    link = (f"https://www.uniprot.org/uniprot/?query={Gen}+capra+hircus&sort=score")
    
    #Obetenmos los datos de la página
    html = requests.get(link)
    #Analizando el archivo de texto formateado en html
    soup = b(html.content, 'html.parser')
    
    #De no encontrar los datos, entonces por en blanco a la lista y agregalo en una fila
    if 'Sorry, no results found for your search term.' in str(soup.text):
        url_Gen=[Gen, '', '' , '', '', '', '', '']
        
   #De no cumplir lo anterior se agrega todo los datos en el Reporte
    else: 
        
        #Buscamos la columna Entry
        #Buscamos la etiqueta <tr> y su clase "entry selected-row"
        Entry=soup.find('tr',class_="entry selected-row").a.text
        
    
        Entryname=soup.find('tr',class_="entry selected-row").find_all('td')[2].text #Buscamos la columna Entryname
        #Encontramos la etiqueta <tr> y su clase "entry selected-row"; extrayendo todos los texto 
        # de la posicion [2] de todas las etiquetas <td>
        url_entry= (f"https://www.uniprot.org/uniprot/{Entry}") #Usamos la informacion obtenida en 'entry', para obtener la información de la siguiente pagina a webscrapear
        html_entry = requests.get(url_entry)
        soup_entry = b(html_entry.content, 'html.parser')
        
        Protein=soup_entry.find('div', id='content-protein').text
        #Encuentramos la primera etiqueta <div> con id 'content-gene' y extraemos el texto
        Gene=soup_entry.find('div', id='content-gene').text
        #Encuentramos la etiqueta y extraemos el texto
        Organism=soup_entry.find('div', id='content-organism').text
        #Encuentramos la primera etiqueta y extraemos el texto de la primera etiqueta <span>
        Status=soup_entry.find('div',id="content-status").span.text
        
        Go_Gen_bio=soup_entry.find('ul', class_="noNumbering biological_process")
        #En caso no encontramos nada en Go_Gen_bio, GO_Biological_process colocamos como una cadena vacia
        if Go_Gen_bio is None:
             GO_Biological_process=' '
         #En caso contrario se encuentre resultados
        else:
            #Si es que llegamos a  econtrar,  empieza a buscar  todas las etiquetas <a> que terminen en 'Display-GO-Term'
            GO_descripcion=Go_Gen_bio.find_all('a', onclick="window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');")
            GO_Biological_process='' #Esta linea  crea una  cadena vacia a la cual se apendizara todo el texto encontrado en GO_descripcion
            #Hicimos un  bucle para unir el texto para cada GO_descripcion encontrado
            for x in GO_descripcion: #Se crea el bucle Go_descripcion y llenara con lo encontrado
                 GO_Biological_process+=x.text+','
        
        url_Gen=[Gen, Entry, Entryname , Protein, Gene, Organism, Status, GO_Biological_process] # Invoca todas las variables asignadas para agregarle los datos 
    encabezados.append(url_Gen)

#### TERCERO
Luego de extraer los datos que renombramos en la variable "encabezados" de la página, estructuramos la información en un archivo .xlsx , el cual se llama **Reporte.xlsx**.
<span style="color:blue;"> 
Nota: El archivo Reporte.xlsx, se encuentra en nuestro repositorio. </span>

In [16]:
info_data=pd.DataFrame(encabezados) #Dataframe
# limpieza de valor "NaN"
info_data.replace(to_replace=[""], value=np.nan, inplace=True) 
info_data.head(5)

,0,1,2,3,4,5,6,7
0,Gen,Entry,Entryname,Protein,Gene,Organism,Status,GO_Biological_process
1,TRNAS-GGA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LOC102173449,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LOC102169333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LOC108636757,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Se esta creando un excel con la dataframe  "data" que nos da el resultado y con header none elimina la primera fila.  
info_data.to_excel("Reporte.xlsx",index=False,header=None) 